In [1]:
# A six-sided die, with numbers written on each of its faces, is placed on the 6-by-6 grid above, 
# in the lower-left (yellow) corner. It then makes a sequence of “moves”. 
# Each move consists of tipping the die into an orthogonally adjacent square within the grid.

# The die starts with a “score” of 0. On the Nth move, 
# its score increases by N times the value of the die facing up after the move. 

# However, the die is only allowed to move into a square if its score after the move matches the value in the square. 
# Also, the die cannot be translated or rotated in place in addition to these moves.

# After some number of moves the die arrives in the upper-right (blue) corner.

# The answer to this puzzle is the sum of values in the unvisited squares from the die’s journey.

In [2]:
numbers = [[57,33,132,268,492,732],
           [81,123,240,443,353,508],
           [186,42,195,704,452,228],
           [-7,2,357,452,317,395],
           [5,23,-4,592,445,620],
           [0,77,32,403,337,452]]

In [3]:
# if movement on x-axis 
def get_new_dice_config(die_config, die_coordi, new_coordi_for_possible_values):

    horizontal = die_config[0].copy()
    vertical = die_config[1].copy()
    
    previous_positionX = die_coordi[0]
    previous_positionY = die_coordi[1]

    current_positionX = new_coordi_for_possible_values[0]
    current_positionY = new_coordi_for_possible_values[1]
    
    new_horizontal = []
    new_vertical = []  
    
    if previous_positionY > current_positionY:
        for i in horizontal:
            if i == horizontal[-1]:
                new_horizontal.insert(0, i)
            else:
                new_horizontal.insert((horizontal.index(i) + 1), i)
        horizontal = new_horizontal
        vertical[1] = horizontal[1] 
        
    elif previous_positionY < current_positionY:
        for i in horizontal:
            new_horizontal.insert((horizontal.index(i) - 1), i)
        horizontal = new_horizontal
        vertical[1] = horizontal[1] 

    elif previous_positionX < current_positionX:
        for i in vertical:
            new_vertical.insert((vertical.index(i) - 1), i)
        vertical = new_vertical
        horizontal[1] = vertical[1]
        vertical_pop = vertical.pop(-1)
        horizontal_pop = horizontal.pop(-1)
        horizontal.append(vertical_pop)
        vertical.append(horizontal_pop)

    elif previous_positionX > current_positionX:
        for i in vertical:
            if i == vertical[-1]:
                new_vertical.insert(0, i)
            else:
                new_vertical.insert((vertical.index(i) + 1), i)
        vertical = new_vertical
        horizontal[1] = vertical[1]
        vertical_pop = vertical.pop(0)
        horizontal_pop = horizontal.pop(-1)
        horizontal.append(vertical_pop)
        vertical.insert(0, horizontal_pop)
        
    if len(set(horizontal)) < 4:
        return horizontal
    if len(set(vertical)) < 3:
        return vertical
    new_die_config = [horizontal, vertical]

    return new_die_config

In [4]:
def get_possible_values_from_coordi(li):
    x, y = li
    possible_values = {}
    if x+1 <= 5:
        possible_values[str(numbers[x+1][y])+'p1'] = [x+1, y]
    if y+1 <= 5:    
        possible_values[str(numbers[x][y+1])+'p2'] = [x, y+1]
    if x-1 >= 0:
        possible_values[str(numbers[x-1][y])+'p3'] = [x-1, y]
    if y-1 >= 0:
        possible_values[str(numbers[x][y-1])+'p4'] = [x, y-1]
    return possible_values       

In [5]:
def get_opposite_face(current_touching_face):
    if current_touching_face == 'n1':
        return 'n3'
    if current_touching_face == 'n2':
        return 'n4'
    if current_touching_face == 'n3':
        return 'n1'
    if current_touching_face == 'n4':
        return 'n2'
    if current_touching_face == 'n5':
        return 'n6'
    if current_touching_face == 'n6':
        return 'n5'

In [6]:
inital_values_of_faces = {'n1':None, 'n2':None, 'n3':None, 'n4':None, 'n5':None, 'n6':None}

In [7]:
parent_list = []
die_config = [['n4','n1','n2','n3'], ['n5', 'n1', 'n6']]

In [8]:
variable_for_each_iteration = [0, 0, [5,0], die_config, inital_values_of_faces, [[5,0]]]
parent_list.append(variable_for_each_iteration)

In [9]:
while parent_list:
    child_list = []
    for i in parent_list:
#       Unpacking
        current_move = i[0]
        current_sum = i[1]
        die_coordi = i[2]
        die_config = i[3]
        faces_to_numbers = i[4]
        positions_visited = i[5]

        possible_values = get_possible_values_from_coordi(die_coordi)
#       possible_values = numbers:coordinates 
        for values in possible_values:
                possbile_new_coordinates = possible_values[values]
                new_die_config = get_new_dice_config(die_config, die_coordi, possbile_new_coordinates)
                new_die_touching_face = new_die_config[0][1]
                
                opposite_face = get_opposite_face(new_die_touching_face)
                new_move_count = current_move + 1
                new_faces_to_numbers = faces_to_numbers.copy()

                if new_faces_to_numbers[opposite_face] == None:
                        new_faces_to_numbers[opposite_face] = (int(values[:-2])-current_sum)/new_move_count 
                new_sum = current_sum + new_faces_to_numbers[opposite_face]*new_move_count
                
                if new_sum == int(values[:-2]):
                    new_positions_visited = positions_visited.copy()
                    new_positions_visited.append(possbile_new_coordinates)   
                    child_list.append([new_move_count, new_sum, possbile_new_coordinates, new_die_config,
                                       new_faces_to_numbers, new_positions_visited])
                    if new_sum == 732:
                        break
            
                elif new_sum != values:
                    continue
                else:
                    print("logic error")

    if child_list == []:
        break
    parent_list = child_list   

In [10]:
repeated_removed_list = []
total_places_visited = 0
for i in parent_list[0][-1]:
    if i not in repeated_removed_list:
        total_places_visited += numbers[i[0]][i[1]]
        repeated_removed_list.append(i)

In [11]:
total_of_all_places = 0
for i in numbers:
    for j in i:
        total_of_all_places += j
total_of_all_places

9767

In [12]:
total_of_all_places - total_places_visited

1935